### Linear Regression Documentation Example

Let's walk through the steps of the official documentation example. Doing this will help your ability to read from the documentation, understand it, and then apply it to your own problems (the upcoming Consulting Project).

In [2]:
!pip install pyspark py4j

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
from pyspark.sql import SparkSession

In [4]:
spark=SparkSession.builder.appName('lrex').getOrCreate()

In [5]:
from pyspark.ml.regression import LinearRegression

In [38]:
# Load training data
training=spark.read.format('libsvm').load('/content/sample_data/sample_linear_regression_data.txt')

Interesting! We haven't seen libsvm formats before. In fact the aren't very popular when working with datasets in Python, but the Spark Documentation makes use of them a lot because of their formatting. Let's see what the training data looks like:

In [9]:
training.show()

+-------------------+--------------------+
|              label|            features|
+-------------------+--------------------+
| -9.490009878824548|(10,[0,1,2,3,4,5,...|
| 0.2577820163584905|(10,[0,1,2,3,4,5,...|
| -4.438869807456516|(10,[0,1,2,3,4,5,...|
|-19.782762789614537|(10,[0,1,2,3,4,5,...|
| -7.966593841555266|(10,[0,1,2,3,4,5,...|
| -7.896274316726144|(10,[0,1,2,3,4,5,...|
| -8.464803554195287|(10,[0,1,2,3,4,5,...|
| 2.1214592666251364|(10,[0,1,2,3,4,5,...|
| 1.0720117616524107|(10,[0,1,2,3,4,5,...|
|-13.772441561702871|(10,[0,1,2,3,4,5,...|
| -5.082010756207233|(10,[0,1,2,3,4,5,...|
|  7.887786536531237|(10,[0,1,2,3,4,5,...|
| 14.323146365332388|(10,[0,1,2,3,4,5,...|
|-20.057482615789212|(10,[0,1,2,3,4,5,...|
|-0.8995693247765151|(10,[0,1,2,3,4,5,...|
| -19.16829262296376|(10,[0,1,2,3,4,5,...|
|  5.601801561245534|(10,[0,1,2,3,4,5,...|
|-3.2256352187273354|(10,[0,1,2,3,4,5,...|
| 1.5299675726687754|(10,[0,1,2,3,4,5,...|
| -0.250102447941961|(10,[0,1,2,3,4,5,...|
+----------

This is the format that Spark expects. Two columns with the names "label" and "features".

The "label" column then needs to have the numerical label, either a regression numerical value, or a numerical value that matches to a classification grouping. Later on we will talk about unsupervised learning algorithms that by their nature do not use or require a label.

The feature column has inside of it a vector of all the features that belong to that row. Usually what we end up doing is combining the various feature columns we have into a single 'features' column using the data transformations we've learned about.

Let's continue working through this simple example!

In [39]:
# These are the default values for the featuresCol, labelCol, predictionCol
lr=LinearRegression(featuresCol='features',labelCol='label',predictionCol='prediction')
# You could also pass in additional parameters for regularization, do the reading 
# in ISLR to fully understand that, after that its just some simple parameter calls.
# Check the documentation with Shift+Tab for more info!

In [40]:
# Fit the model
lrModel = lr.fit(training)

In [41]:
# Print the coefficients and intercept for linear regression
print("Coefficients: {}".format(str(lrModel.coefficients))) # For each feature...
print('\n')
print("Intercept:{}".format(str(lrModel.intercept)))

Coefficients: [0.0073350710225801715,0.8313757584337543,-0.8095307954684084,2.441191686884721,0.5191713795290003,1.1534591903547016,-0.2989124112808717,-0.5128514186201779,-0.619712827067017,0.6956151804322931]


Intercept:0.14228558260358093


There is a summary attribute that contains even more info!

In [42]:
# Summarize the model over the training set and print out some metrics
training_summary=lrModel.summary

Lots of info, here are a few examples:

In [44]:
training_summary.residuals.show()
print("RMSE: {}".format(training_summary.rootMeanSquaredError))
print("r2: {}".format(training_summary.r2))

+-------------------+
|          residuals|
+-------------------+
|-11.011130022096554|
| 0.9236590911176538|
|-4.5957401897776675|
|  -20.4201774575836|
|-10.339160314788181|
|-5.9552091439610555|
|-10.726906349283922|
|  2.122807193191233|
|  4.077122222293811|
|-17.316168071241652|
| -4.593044343959059|
|  6.380476690746936|
| 11.320566035059846|
|-20.721971774534094|
| -2.736692773777401|
| -16.66886934252847|
|  8.242186378876315|
|-1.3723486332690233|
|-0.7060332131264666|
|-1.1591135969994064|
+-------------------+
only showing top 20 rows

RMSE: 10.16309157133015
r2: 0.027839179518600154


## Train/Test Splits

But wait! We've commited a big mistake, we never separated our data set into a training and test set. Instead we trained on ALL of the data, something we generally want to avoid doing. Read ISLR and check out the theory lecture for more info on this, but remember we won't get a fair evaluation of our model by judging how well it does again on the same data it was trained on!

Luckily Spark DataFrames have an almost too convienent method of splitting the data! Let's see it:

In [22]:
all_data=spark.read.format('libsvm').load('/content/sample_data/sample_linear_regression_data.txt')

In [45]:
# Pass in the split between training/test as a list.
# No correct, but generally 70/30 or 60/40 splits are used. 
# Depending on how much data you have and how unbalanced it is.
train_data,test_data=all_data.randomSplit([0.7,0.3])

In [46]:
train_data.describe().show()

+-------+--------------------+
|summary|               label|
+-------+--------------------+
|  count|                 356|
|   mean|-0.12517698043780118|
| stddev|  10.174809440911536|
|    min| -28.571478869743427|
|    max|   27.78383192005107|
+-------+--------------------+



In [47]:
test_data.describe().show()

+-------+-------------------+
|summary|              label|
+-------+-------------------+
|  count|                145|
|   mean| 1.1949262410727393|
| stddev|  10.63857491983645|
|    min|-28.046018037776633|
|    max| 26.903524792043335|
+-------+-------------------+



Now we only train on the train_data

In [48]:
correct_model=lr.fit(train_data)

Now we can directly get a .summary object using the evaluate method:

In [49]:
test_results=correct_model.evaluate(test_data)

In [50]:
test_results.residuals.show()
print("RMSE: {}".format(test_results.rootMeanSquaredError))

+-------------------+
|          residuals|
+-------------------+
|-28.479244065165986|
|-27.294435459451353|
|-21.601926224267665|
| -23.87370915122182|
|-25.502402864884452|
| -17.77337988194499|
|-16.182354628975567|
|-19.352831911559026|
|-15.910242936109684|
| -16.00888682382688|
|-12.202054463839504|
|-12.482328523185176|
|-14.303512996697824|
|-15.451889364212265|
| -7.783261426580812|
| -12.43575417868677|
|-15.165068868905413|
| -13.84228470752555|
|-11.930012928977721|
|-11.503737455946338|
+-------------------+
only showing top 20 rows

RMSE: 10.73182455388563


In [34]:
unlabeled_data=test_data.select('features')

In [35]:
unlabeled_data.show()

+--------------------+
|            features|
+--------------------+
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
+--------------------+
only showing top 20 rows



Well that is nice, but realistically we will eventually want to test this model against unlabeled data, after all, that is the whole point of building the model in the first place. We can again do this with a convenient method call, in this case, transform(). Which was actually being called within the evaluate() method. Let's see it in action:

In [36]:
predictions=correct_model.transform(unlabeled_data)

In [37]:
predictions.show()

+--------------------+--------------------+
|            features|          prediction|
+--------------------+--------------------+
|(10,[0,1,2,3,4,5,...| -3.6667213643743843|
|(10,[0,1,2,3,4,5,...|  2.8472421684362996|
|(10,[0,1,2,3,4,5,...|  0.8991762649314347|
|(10,[0,1,2,3,4,5,...| -2.6049205393910917|
|(10,[0,1,2,3,4,5,...|  0.6249318558130322|
|(10,[0,1,2,3,4,5,...|  0.6125585831032094|
|(10,[0,1,2,3,4,5,...| -0.3219028744369153|
|(10,[0,1,2,3,4,5,...|  1.6865663827525204|
|(10,[0,1,2,3,4,5,...| -2.3095278517266706|
|(10,[0,1,2,3,4,5,...| -1.6987796080655002|
|(10,[0,1,2,3,4,5,...|   3.376686512062108|
|(10,[0,1,2,3,4,5,...|-0.16829671587186107|
|(10,[0,1,2,3,4,5,...|  0.3104552641160903|
|(10,[0,1,2,3,4,5,...| -3.9526818406576916|
|(10,[0,1,2,3,4,5,...|   1.067537989906657|
|(10,[0,1,2,3,4,5,...|   1.734979992491562|
|(10,[0,1,2,3,4,5,...|   2.014878026346038|
|(10,[0,1,2,3,4,5,...|  -1.530251177222366|
|(10,[0,1,2,3,4,5,...| 0.36652016245677377|
|(10,[0,1,2,3,4,5,...| -0.758092

Okay, so this data is a bit meaningless, so let's explore this same process with some data that actually makes a little more intuitive sense!